## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**СОДЕРЖАНИЕ:**


[**Шаг 1. Откройте файл с данными и изучите общую информацию.**](#step1)

[**Шаг 2. Предобработка данных**](#step2)

[Обработка пропусков](#obrabotkapropuskov)

[Замена типа данных](#tipdannykh)

[Обработка дубликатов](#duplicaty)

[Лемматизация](#lemma)

[Категоризация данных](#category)

[**Шаг 3. Ответы на вопросы**](#answers)

[**Шаг 4. Общий вывод**](#step4)

### Шаг 1. Откройте файл с данными и изучите общую информацию. 
<a name="step1" />

Прочитаем файл 'data.csv' и сохраним его в переменной data.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк на экран.

In [2]:
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

Получим общую информацию о таблице.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов. В них представленны данные трех типов: float, integer и object.

  children — количество детей в семье
  
  days_employed — общий трудовой стаж в днях
  
  dob_years — возраст клиента в годах
  
  education — уровень образования клиента
  
  education_id — идентификатор уровня образования
  
  family_status — семейное положение
  
  family_status_id — идентификатор семейного положения
  
  gender — пол клиента
  
  income_type — тип занятости
  
  debt — имел ли задолженность по возврату кредитов
  
  total_income — ежемесячный доход
  
  purpose — цель получения кредита
  
  Что сразу бросается в глаза?
  
1. В столбцах days_employed и total_income имеет меньшее количество значений в строках, что говорит о наличии пропущенных значений. 

2. В столбце days_employed мы видим отрицательные значения. С причиной этого предстоит разобраться, т.к. трудовой стаж в днях не может быть отрицательным числом.

3. Столбец  education содержит одинаковые слова, но напечатанные буквами разного регистра.
4. В столбце purpose содержатся различные формулировки одних и тех же целей.

Для полноты картины вызовем еще один метод, дающий сводку основной информации о таблице.

In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы видим следующие артефакты: отрицательное количество детей, возраст заемщика = о лет.

### Вывод

В строках таблицы содержится достаточное количество информации, чтобы разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
Необходимо сделать объемную работу по предобработке данных, чтобы провести анализ: избавиться от пропусков, выяснить причину появления отрицательных значений и устранить ее, проверить таблицу на наличие дубликатов, привести слова к одному регистру, применить лемматизацию.
Для исследования надежности заемщиков является важным ответить на вопросы:
  - Есть ли зависимость между наличием детей и возвратом кредита в срок?
  - Есть ли зависимость между семейным положением и возвратом кредита в срок?
  - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  - Как разные цели кредита влияют на его возврат в срок?

### Шаг 2. Предобработка данных
<a name="step2" />

### Обработка пропусков
<a name="obrabotkapropuskov" />

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
print(data.isnull().sum()) 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Мы видим, что в столбцах days_employed и total_income по одинаковому количеству пропусков. Помимо пропусков в столбце с общим трудовым стажем есть и другая проблема - отрицательные значения. Но в нашем случае, для решения поставленных задач, нет необходимости в применении данных этого столбца для анализа. Поэтому считаю допустимым заменить эти значения на ноль. 
Причиной пропусков мог послужить неправильный сбор данных, например, заполняемые поля не являются обязательными.Так же данные могут собираться из разных отделений банка и заполняться в разных форматах. 
Так же стоит обратить внимание разработчиков на валидацию данных, для того, чтобы можно было использовать эти данные в дальнейших исследованиях.

Сумма ежемесечного дохода - важный показатель для нашего исследования. И так как количество пропусков составляет более 10 % от общего количества строк, то нам будет недостаточно просто убрать пропуски, необходимо заменить пустые значения медианным значением по столбцу.

In [6]:

data.loc[data['total_income'] == 0, ['total_income']].count()

total_income    0
dtype: int64

In [7]:
data['total_income'].min()

20667.26379327158

Чтобы не лишиться строк с важными данными, заполним значения NaN в столбце 'days_employed' нулями.
Для этого лучше всего использовать метод fillna(), где в качестве аргумента выступает заменитель отсутствующих значений.

In [8]:
data['days_employed'] = data['days_employed'].fillna(0)

Проверим, остались ли пропуски в столбце 'days_employed'.

In [9]:
print(data['days_employed'].isnull().sum())

0


Пропусков в столбце days_employed не осталось.
Теперь перейдем к обработке данных столбца с суммами ежемесячного дохода.
Заменить пропуски на ноль, в данном случае будет не корректо, т.к. по предполагаемая причина пропусков - ошибка в сборе или выгрузке информации. И т.к. количество строк с пропусками более 10%, мы можем потерять важные данные из других столбцов.
Высчитать среднее значение или медиану в целом по столбцу 'total_income' и заменить пропуски им, тоже считаю не корректным, так как данные могут сильно разниться в зависимости от типа занятости.
Считаю, что наиболее приемлемым вариантом будет высчитать медианы в зависимости от типа занятости и заменить пропуски на медианы соответствующих категорий типа занятости.

Найдем уникальные значения в столбце 'income_type' и высчитаем медианные значения по ним и заменим пропуски  в соответствии с категорией типа данных.

В завершении сделаем проверку на отсутствие пропусков в столбце 'total_income'.

In [10]:
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, median)
    data.loc[(data['total_income'].isnull()) & (data['income_type'] == income_type), 'total_income'] = median
print(data['total_income'].isnull().sum())   

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
0


Заменим пропущенные значения в столбце total_income на медиану значений этого же столбца.

В столбце dob_years имеются нулевые значения, которых быть не должно, т.к. возраст заемщика не может быть ниже 18 лет. Посмотрим, сколько всего строк с нулевым значением.

In [11]:
data.loc[data['dob_years'] == 0, ['dob_years']].count()

dob_years    101
dtype: int64

Посчитаем средний возраст по столбцу dob_years

In [12]:
data['dob_years'].mean()

43.29337979094077

Заменим значение 0 в столбце dob_years на 43 (средний возраст)

In [13]:
data.loc[data['dob_years'] == 0, 'dob_years'] = 43

Проверим, остались ли еще нули в столбце 'dob_years'.

In [14]:
data.loc[data['dob_years'] == 0, ['dob_years']].count()

dob_years    0
dtype: int64

Ну и на последок, разберемся с отрицательным количеством детей в столбце children.

проверим, какие уникальные значения есть столбце children.

In [15]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


В столбце 'children' содержится число со знаком минус. Число детей отрицательным быть не может. Проверим сколько таких значений:

In [16]:
print(data.loc[data['children'] == -1, ['children']].count())

children    47
dtype: int64


Мы получили всего 47 строк. Есть вероятность, что знак минус был поставлен по неосмотрительности, поэтому считаю логичным заменить в этих строках число детей на 1.

In [17]:
data.loc[(data['children']== -1), 'children'] = 1


Проверим не осталось ли отрицательных значений в столбце 'children'

In [18]:
print(data['children'].unique())

[ 1  0  3  2  4 20  5]


Из списка значений по количеству детей, так же выделяется число 20. Проверим, сколько таких значений.

In [19]:
print(data.loc[data['children'] == 20, ['children']].count())

children    76
dtype: int64


Гипотетически, в семье может быть 20 детей, но мы не видим в списке похожих значений. Поэтому, высока вероятность, что здесь тоже закралась ошибка. Например, число выводилось в виде "2,0", а при экспорте данных знак запятой пропал.А возможно эта строка была задублирована 75 раз. Проверим эту гепотезу после удаления дубликатов.

### Вывод

Мы не стали тратить время на обработку данных столбца days_employed, данные которого не несут нам полезной информации для решения поставленных задач и заменили пропуски в нем на нулевые значения. Но есть предложение передать информацию разработчикам о том, что валидация данных проходит не корректно.
От пропусков в столбце total_income мы избавились путем замены пропущенных значений на медианный доход заемщиков в зависимости от его типа занятости. 
Так же мы заменили некорректные нулевые значения в столбце 'dob_years' на средний возраст.
В столбце children исправлено отрицательное количество детей, предположительно появившееся в следствие ошибки. Допускаю, что из-за человеческого фактора. Так же в этом столбце обнаружен такой артефакт , как 20 детей, это значение повторяется 76 раз. Но, так как нет необходимости выяснять какое количество детей влияет на возвращение кредитов в срок, а важно лишь их наличие, исходные данные оставим прежними. 

### Замена типа данных
<a name="tipdannykh" />

В таблице два столбца имеют тип данных float: days_employed и total_income.Значения days_employed нам не требуются для решения поставленной задачи.
А значения столбца total_income имеют право быть типа float, т.к. это сумма  в рублях, и значения могут иметь десятичные и сотые доли (копейки).Но для удобства будущих расчетов, все же переведем значения столбца 'total_income' в целочисленные.

In [20]:
data['total_income'] = data['total_income'].astype('int')

### Вывод

Тип данных столбцов соответствует содержимым данным, за исключением столбца days_employed. Значения данного столбца нам не требуются для решения поставленной задачи, поэтому просто передадим информацию об этом явлении разрабочикам.

### Обработка дубликатов
<a name="duplicaty" />

В столбце education мы видим одинаковые значения, отличающиеся регистром. Это может помешать поиску дубликатов. Поэтому необходимо, перевести его к одному регистру.

In [21]:
data['education_lowercase'] = data['education'].str.lower()
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Проверим, сколько всего дубликатов в таблице, прежде, чем перейти к их удалению.

In [22]:
data.duplicated(subset=['children', 'days_employed', 'dob_years', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'education_lowercase'], keep=False).sum()

137

Всего 137 дубликатов.Это меньше 1 % от общего количества строк таблицы.

Применим метод drop_duplicates, удаляющий дупликаты построчно. У нас достаточно большой перечень столбцов, нам нужно будет перечислить их все, за исключением education. Выведем на экран названия столбцов таблицы в виде списка, и используем его при перечислении столбцов в строке код, при удалении дубликатов, удалим из него вручную название  education.

In [23]:
print(list(data))

['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'education_lowercase']


In [24]:
data.drop_duplicates(subset=['children', 'days_employed', 'dob_years', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'education_lowercase'], keep=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее


В таблице было 21525 строк, стало 21388, то есть было удалено 137 дубликатов.

Вернемся к достаточно странному значению "20 детей". Посмотрим сколько строк в столбце children содержит это значение после удаления дубликатов.

In [25]:
print(data.loc[data['children'] == 20, ['children']].count())

children    76
dtype: int64


Для дальнейших расчетов, примем число 20 детей, как ошибочное, и заменим на 2 (большая вероятность, что был проставлен лишний ноль.

In [26]:
data.loc[(data['children']== 20), 'children'] = 2

### Вывод


Таблица очищена от дубликатов в количестве 137, что составляет менее 1 % от количества строк таблицы.


### Лемматизация
<a name="lemma" />

Выгрузим библиотеку pymystem3

In [27]:
from pymystem3 import Mystem
m=Mystem()
 


Лемматизируем столбец 'purpose'

In [28]:
data['lemma_purpose']=data['purpose'].apply(m.lemmatize)

Для подсчёта числа упоминаний лемм в тексте вызываем специальный контейнер Counter из модуля collections.

In [29]:
from collections import Counter
unique_purpose = ' '.join(data['purpose'].unique())
lemmas = m.lemmatize(unique_purpose)
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Для полноты понимания общей картины и для разбивки целей кредита на группы, посмотрим список уникальных значений в столбце 'purpose'

In [30]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Теперь можно выделить следующие категории: 'на покупку недвижимости', 'на свадьбу', 'на автомобиль', 'на образование', 'на ремонт'. На случай, если в данные попадет артефакт ,добавим значение: 'на другое'. Мы видим значение 'ремонт жилью'. Учтем то, что в нем два слова, которые мы выделили в леммы, при написании функции. Слово ремонт имеет первостепенную роль в данном контексте, поэтому поместим это условие в функции, распределяющей цели кредита, в начало.

In [31]:
def change_purpose(lemma):
    if 'ремонт' in lemma:
        return 'на ремонт жилья'
    if ('жилье' in lemma) or ('недвижимость' in lemma):
        return 'на покупку недвижимости'
    if 'свадьба' in lemma:
        return 'на свадьбу'
    if 'автомобиль' in lemma:
        return 'на автомобиль'
    if 'образование' in lemma:
        return 'на образование'
    return 'на другое'

Заменим значения в столбце 'lemma_purpose' после выполнения функции.

In [32]:
data['lemma_purpose']=data['lemma_purpose'].apply(change_purpose)

Проверим, что все цели кредита учтены.

In [33]:
data['lemma_purpose'].unique()

array(['на покупку недвижимости', 'на автомобиль', 'на образование',
       'на свадьбу', 'на ремонт жилья'], dtype=object)

Посмотрим, как выглядит таблица после лемматизации.

In [34]:
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,lemma_purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее,на покупку недвижимости
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее,на автомобиль
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее,на покупку недвижимости
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее,на автомобиль
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,среднее,на автомобиль


### Вывод

После выполнения лемматизации данных в столбце 'purpose', мы получили новый столбец 'lemma_purpose', в котором значения распределены на 5 видов значений , которые будет удобно группировать:'на покупку недвижимости', 'на автомобиль', 'на образование','на свадьбу', 'на ремонт жилья'.

### Категоризация данных
<a name="category" />

Начнем с категоризации данных по столбцу total_income. Прежде чем, это сделать , нужно понимать, какое минимальное и максимальное значение по столбцу.

In [35]:
data['total_income'].min()

20667

In [36]:
data['total_income'].max()

2265604

Теперь разобьем все значения на квантили.

In [37]:
data['total_income'].quantile([.25, .50, .75])


0.25    107798.0
0.50    142594.0
0.75    195549.0
Name: total_income, dtype: float64

Напишем функцию, которая будет выбирать из диапазона значений сумму дохода и присваивать числу категорию "низкий", "средний", "выше среднего" или "высокий".

In [38]:
def group_income(income):
    if 0 < income <= 107798:
        return 'низкий'
    if 107799 <= income <= 142594:
        return 'средний'
    if 142595 <= income <= 195549:
        return 'выше среднего'
    if  income >=195549:
        return 'высокий'

Создадим новый столбец group_income, и применив функцию group_income и метод apply, заполним его значениями категорий, в соответствии с уровнем дохода.

In [39]:
data['group_income'] = data['total_income'].apply(group_income)

Сделаем проверку и посмотрим, все ли категории присутствуют в таблице.

In [40]:
data['group_income'].unique()

array(['высокий', 'средний', 'выше среднего', 'низкий'], dtype=object)

Проверим значения в новом столбце group_income в таблице.

In [41]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,lemma_purpose,group_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,на покупку недвижимости,высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,на автомобиль,средний
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,на покупку недвижимости,выше среднего
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,на образование,высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,на свадьбу,выше среднего


Так же проведем категоризацию по столбцу Children. Разделим заемщиков на три категории "малодетные семьи", "многодетные семьи" и "нет детей".

In [42]:
def group_children (children):
    if 0 < children <=2:
        return 'малодетные семьи'
    if  3 <= children <=20:
        return 'многодетные семьи'
    else:
        return 'нет детей'

Создадим новый столбец с категориями "малодетные семьи", "многодетные семьи" и "нет детей", назовем его group_children.

In [43]:
data['group_children'] = data['children'].apply(group_children)

Ну и в заключении еще раз посмотрим на получившуюся таблицу и проверим значения в новом столбце group_children.

In [44]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,lemma_purpose,group_income,group_children
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,на покупку недвижимости,высокий,малодетные семьи
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,на автомобиль,средний,малодетные семьи
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,на покупку недвижимости,выше среднего,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,на образование,высокий,многодетные семьи
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,на свадьбу,выше среднего,нет детей


### Вывод

Категоризация выполнена по всем необходимым для анализа столбцам: по цели кредита, по наличию детей , по величине дохода.

Создадим новую таблицу, в которой будут содежаться только небходимые для анализа данные.

In [45]:
data_for_analize = data[['family_status','income_type','total_income','lemma_purpose','group_income','group_children','children','debt']].copy()

In [46]:
data_for_analize.head()

,family_status,income_type,total_income,lemma_purpose,group_income,group_children,children,debt
0,женат / замужем,сотрудник,253875,на покупку недвижимости,высокий,малодетные семьи,1,0
1,женат / замужем,сотрудник,112080,на автомобиль,средний,малодетные семьи,1,0
2,женат / замужем,сотрудник,145885,на покупку недвижимости,выше среднего,нет детей,0,0
3,женат / замужем,сотрудник,267628,на образование,высокий,многодетные семьи,3,0
4,гражданский брак,пенсионер,158616,на свадьбу,выше среднего,нет детей,0,0


### Шаг 3. Ответьте на вопросы
<a name="answers" />

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим двумерную сводную таблицу, сгруппированную по столбцу group_children и debt

In [47]:
report = data.pivot_table(index = 'group_children', columns = 'debt', values = 'education', aggfunc = 'count')  
report

debt,0,1
group_children,,
малодетные семьи,6349,647
многодетные семьи,349,31
нет детей,13086,1063


Назовем столбцы сгруппированного столбца debt: 'no_debt', 'debt'

In [48]:
report.columns = ['no_debt', 'debt']

Добавим столбец с названием %, который показывает процент людей, имевших задолженность по возврату кредитов.

In [49]:
report['%']=report['debt']/(report['debt']+report['no_debt'])

Зададим необходимый формат значениям.

In [50]:
report.style.format({'%':'{:.2%}','no_debt':'{:.0f}'}) 

,no_debt,debt,%
group_children,,,
малодетные семьи,6349,647,9.25%
многодетные семьи,349,31,8.16%
нет детей,13086,1063,7.51%


### Вывод

Клиенты банка, имеющие 1 или 2 детей  имели задолженность по погашению кредита, чаще, чем клиенты не имеющие детей на 1.74 % .

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

По аналогии построим двумерную сводную таблицу, но в качестве index  возьмем столбец 'family_status'

In [51]:
report = data.pivot_table(index = 'family_status', columns = 'debt', values = 'education', aggfunc = 'count')  
report

debt,0,1
family_status,,
Не женат / не замужем,2539,274
в разводе,1110,85
вдовец / вдова,897,63
гражданский брак,3789,388
женат / замужем,11449,931


In [52]:
report.columns = ['no_debt', 'debt']

In [53]:
report['%']=report['debt']/(report['debt']+report['no_debt'])

In [54]:
report.style.format({'%':'{:.2%}','no_debt':'{:.0f}'}) 

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2539,274,9.74%
в разводе,1110,85,7.11%
вдовец / вдова,897,63,6.56%
гражданский брак,3789,388,9.29%
женат / замужем,11449,931,7.52%


### Вывод

Наименьший процент наличия задолженности по возврату имеют клиенты со статусом: вдовец/вдова. Наибольший - клиенты, имеющий статус: не женат/не замужем. Разница между ними составляет 3.18%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

По аналогии построим двумерную сводную таблицу, но в качестве index возьмем столбец 'group_income'

In [55]:
report = data.pivot_table(index = 'group_income', columns = 'debt', values = 'education', aggfunc = 'count')  
report

debt,0,1
group_income,,
высокий,4995,386
выше среднего,4794,445
низкий,4955,427
средний,5040,483


In [56]:
report.columns = ['no_debt', 'debt']

In [57]:
report['%']=report['debt']/(report['debt']+report['no_debt'])

In [58]:
report.style.format({'%':'{:.2%}','no_debt':'{:.0f}'}) 

,no_debt,debt,%
group_income,,,
высокий,4995,386,7.17%
выше среднего,4794,445,8.49%
низкий,4955,427,7.93%
средний,5040,483,8.75%


### Вывод

Клиенты с высоким доходом чаще всех погашают кредиты в срок, а клиенты со средним доходом - реже, чем все остальные. Разница между ними 1.58%

- Как разные цели кредита влияют на его возврат в срок?

По аналогии построим двумерную сводную таблицу, но в качестве index возьмем столбец 'lemma_purpose'

In [59]:
report = data.pivot_table(index = 'lemma_purpose', columns = 'debt', values = 'education', aggfunc = 'count')  
report

debt,0,1
lemma_purpose,,
на автомобиль,3912,403
на образование,3652,370
на покупку недвижимости,9481,747
на ремонт жилья,577,35
на свадьбу,2162,186


In [60]:
report.columns = ['no_debt', 'debt']

In [61]:
report['%']=report['debt']/(report['debt']+report['no_debt'])

In [62]:
report.style.format({'%':'{:.2%}','no_debt':'{:.0f}'}) 

,no_debt,debt,%
lemma_purpose,,,
на автомобиль,3912,403,9.34%
на образование,3652,370,9.20%
на покупку недвижимости,9481,747,7.30%
на ремонт жилья,577,35,5.72%
на свадьбу,2162,186,7.92%


### Вывод

Чаще всего погашают в срок кредиты, взятые на ремонт жилья. Реже всего - на покупку автомобиля. Разница между ними - 3.62%

### Шаг 4. Общий вывод
<a name="step4" />

Я бы распределила критерии, по которым идет оценка "идеального заемщика" по следующему порядку(опираясь на разницу между максимальным % задолженности и минимальным) :
1.Цель кредита (разница 3.62%). Отдавать предпочтение  в таком порядке: на ремонт жилья, на покупку недвижимости, на свадьбу и в последнюю очередь на образование и на покупку автомобиля)
2.Семейное положение (разница 3.18%) Отдавать предпочтение заемщикам в таком порядке: в начале вдовец/вдова, в разводе, женат/замужем, гражданский брак и в последнюю очередь не женат/ не замужем.
3.Наличие детей (разница 1.74 %). Отдавать предпочтение бездетным, во вторую очередь многодетным семьям и в третью - малодетным.
4.Уровень дохода (разница 1.58%). Отдавать предпочтение в таком порядке:высокий доход, низкий доход, выше среднего, средний.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.